In [1]:
from board import Board
from player import Player
from gamestate import GameState
from time import time
import math

In [2]:
def utility(board):
    return int(board.get_game_state()._value_ * 10000 * board.get_rows() * board.get_cols() / board.get_number_of_moves())

def actions(board):
    available_actions = []
    for i in range(board.get_cols()):
        if not board.is_column_full(i):
            available_actions.append(i)
    # print(available_actions)
    return available_actions

def minimax_A(board, table):
    if board in table.keys():
        return table[board]
    elif board.get_game_state() != GameState.IN_PROGRESS:
        info = (utility(board), None)
        table[board] = info
        return info
    elif board.get_player_to_move_next() == Player.MAX:
        v = -math.inf
        best_move = None
        for action in actions(board):
            child_board = board.make_move(action)
            child_info = minimax_A(child_board, table)
            v2 = child_info[0]
            if v2 > v:
                v = v2
                best_move = action
        info = (v, best_move)
        table[board] = info
        return info
    elif board.get_player_to_move_next() == Player.MIN:
        v = math.inf
        best_move = None
        for action in actions(board):
            child_board = board.make_move(action)
            child_info = minimax_A(child_board, table)
            v2 = child_info[0]
            if v2 < v:
                v = v2
                best_move = action
        info = (v, best_move)
        table[board] = info
        return info

def minimax_B(board, alpha, beta, table):
    if board in table.keys():
        return table[board]
    elif board.get_game_state() != GameState.IN_PROGRESS:
        info = (utility(board), None)
        table[board] = info
        return info
    elif board.get_player_to_move_next() == Player.MAX:
        v = -math.inf
        best_move = None
        for action in actions(board):
            child_board = board.make_move(action)
            child_info = minimax_B(child_board, alpha, beta, table)
            v2 = child_info[0]
            if v2 > v:
                v = v2
                best_move = action
        info = (v, best_move)
        table[board] = info
        return info
    elif board.get_player_to_move_next() == Player.MIN:
        v = math.inf
        best_move = None
        for action in actions(board):
            child_board = board.make_move(action)
            child_info = minimax_B(child_board, alpha, beta, table)
            v2 = child_info[0]
            if v2 < v:
                v = v2
                best_move = action
        info = (v, best_move)
        table[board] = info
        return info

def main():
    r = int(input("Enter rows: "))
    c = int(input("Enter columns: "))
    inarow = int(input("Enter number in a row to win: "))
    table = dict()
    board = Board(r, c, inarow)
    start = time()
    result = minimax_A(board, table)
    end = time()
    print("Search completed in %.3f seconds." % (end - start))
    print("Transposition table has %d states." % (len(table.keys())))
    if result[0] == 0:
        print("The game results in a draw with perfect play")
    elif result[0] > 0:
        print("First player has a guaranteed win with perfect play.")
    else:
        print("Second player has a guaranteed win with perfect play.")
    first_player = int(input("Who plays first? 1=human, 2=computer: "))
    players = dict()
    if first_player == 1:
        players["MAX"] = "human"
        players["MIN"] = "computer"
    else:
        players["MAX"] = "computer"
        players["MIN"] = "human"
    while True:
        current_board = Board(r, c, inarow)
        while current_board.get_game_state() == GameState.IN_PROGRESS:
            print(current_board.to_2d_string())
            print("Minimax value for this state: %d, optimal move: %d" % (table[current_board][0], table[current_board][1]))
            next_player = "MAX" if current_board.get_player_to_move_next() == Player.MAX else "MIN"
            print("It is %s's turn!" % (next_player))
            if players[next_player] == "human":
                move = int(input("Enter move: "))
            else:
                move = table[current_board][1]
                print("Computer chooses move: %d" % (move))
            current_board = current_board.make_move(move)
        print(current_board.to_2d_string())
        if current_board.get_game_state() != GameState.TIE:
            winner = "MAX" if current_board.get_game_state() == GameState.MAX_WIN else "MIN"
            print("The winner is %s (%s)" % (winner, players[winner]))
        else:
            print("Draw!")
        again = input("PLay again? (y/n): ")
        if again == "n": break
        


In [15]:
def get_streaks(board):
    streaks = dict()
    for r in range(board.get_rows()):
        for c in range(board.get_cols()):
            if ((c < board.get_cols() - 2 and board.board[r][c] == board.board[r][c+1] == board.board[r][c+2])
                or (r < board.get_rows() - 2 and board.board[r][c] == board.board[r+1][c] == board.board[r+2][c])
                or (r < board.get_cols() - 2 and c < board.get_rows() - 2 and board.board[r][c] == board.board[r+1][c+1] == board.board[r+2][c+2])
                or (r < board.get_rows() - 2 and c > 1 and board.board[r][c] == board.board[r+1][c-1] == board.board[r+2][c-2])):
                return
            
 
def count_in_a_row(board):
    streaks = dict()
    player = board.get_player_to_move_next()._value_
    for r in range(board.get_rows()):
        count = 0
        begin = (r, 0)
        for c in range(board.get_cols() - 1):
            if board.board[r][c] != player:
                continue
            if board.board[r][c+1] == player:
                count += 1
            if board.board[r][c+1] != player and count > 1:               
                streaks[begin] = (r, c)
                begin = (r, c + 1)
                count = 0
    return streaks

def count_in_a_col(board):
    streaks = dict()
    player = board.get_player_to_move_next()._value_
    for c in range(board.get_cols()):
        count = 0
        begin = (0, c)
        for r in range(board.get_rows() - 1):
            if board.board[r][c] != player:
                continue
            if board.board[r+1][c] == player:
                count += 1
            if board.board[r+1][c] != player and count > 1:               
                streaks[begin] = (r, c)
                begin = (r + 1, c)
                count = 0
    return streaks


def count_in_ne_diag(board):
    streaks = dict()
    player = board.get_player_to_move_next()._value_
    for r in range(board.get_rows() - 1):
        count = 0
        begin = (r, 0)
        for c in range(board.get_cols() - 1):
            if board.board[r][c] != player:
                continue
            if board.board[r+1][c+1] == player:
                count += 1
            if board.board[r+1][c+1] != player and count > 1:               
                streaks[begin] = (r, c)
                begin = (r + 1, c + 1)
                count = 0
    return streaks

def count_in_nw_diag(board):
    streaks = dict()
    player = board.get_player_to_move_next()._value_
    for r in range(board.get_rows() - 1):
        count = 0
        begin = (r, 0)
        for c in range(1, board.get_cols()):
            if board.board[r][c] != player:
                continue
            if board.board[r+1][c-1] == player:
                count += 1
            if board.board[r+1][c-1] != player and count > 1:               
                streaks[begin] = (r, c)
                begin = (r + 1, c - 1)
                count = 0
    return streaks

In [20]:
board = Board(5, 5, 4)
board.get_player_to_move_next()._value_

1

In [26]:
def heuristic(board):
    h = 0
    for row in board.board:
        h += get_points(row)
    for col in get_columns(board):
        h += get_points(col)
    for ne_diagonals in get_ne_diagonals(board):
        h += get_points(ne_diagonals)
    for nw_diagonals in get_nw_diagonals(board):
        h += get_points(nw_diagonals)
    return h

def get_points(list):
    count = 1
    points = 0
    for i in range(len(list) - 1):
        if list[i] == list[i + 1]:
            count += 1
        else:
            if count == 2:
                points += 10 * list[i]
            if count == 3:
                points += 30 * list[i]
            count = 1
    if count == 2:
        points += 10 * list[i]
    if count == 3:
        points += 30 * list[i]
    return points
            

def get_ne_diagonals(board):
    diagonals = []
    for r in range(board.get_rows()):
        c = 0
        diagonal = [board.board[r][c]]
        next_r = r + 1
        while next_r < board.get_rows() and c < board.get_cols():
            diagonal.append(board.board[next_r][c + 1])
            c += 1
            next_r += 1
        diagonals.append(diagonal)
    for c in range(board.get_cols()):
        r = 0
        diagonal = [board.board[r][c]]
        next_c = c + 1
        while next_c < board.get_cols() and r < board.get_rows():
            diagonal.append(board.board[r + 1][next_c])
            r += 1
            next_c += 1
        diagonals.append(diagonal)
    return diagonals

def get_nw_diagonals(board):
    diagonals = []
    for r in range(board.get_rows()):
        c = 0
        diagonal = [board.board[r][c]]
        next_r = r - 1
        while next_r >= 0 and c < board.get_cols():
            diagonal.append(board.board[next_r][c + 1])
            c += 1
            next_r -= 1
        diagonals.append(diagonal)
    for c in range(board.get_cols()):
        r = board.get_rows() - 1
        diagonal = [board.board[r][c]]
        next_c = c + 1
        while next_c < board.get_cols() and r >= 0:
            diagonal.append(board.board[r - 1][next_c])
            r -= 1
            next_c += 1
        diagonals.append(diagonal)
    return diagonals

def get_columns(board):
    columns = []
    for c in range(board.get_cols()):
        column = []
        for r in range(board.get_rows()):
            column.append(board.board[r][c])
        columns.append(column)
    return columns

[[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0], [0]]
